In [1]:
# from utils import *

In [2]:
# %%time
# train_ads, train_feeds = pd.read_csv('./data/train/train_data_ads.csv'), pd.read_csv('./data/train/train_data_feeds.csv')
# test_ads, test_feeds = pd.read_csv('./data/test/test_data_ads.csv'), pd.read_csv('./data/test/test_data_feeds.csv')
# train_ads.shape, train_feeds.shape, test_ads.shape, test_feeds.shape

CPU times: user 26.8 s, sys: 2.29 s, total: 29.1 s
Wall time: 29.1 s


((7675517, 35), (3227732, 28), (976058, 34), (369341, 28))

In [3]:
# train_ads = reduce(train_ads)
# train_feeds = reduce(train_feeds)
# test_ads = reduce(test_ads)
# test_feeds = reduce(test_feeds)
# train_ads.shape, train_feeds.shape, test_ads.shape, test_feeds.shape

100%|██████████████████████████████████████████| 28/28 [00:00<00:00, 293.04it/s]


((7675517, 35), (3227732, 28), (976058, 34), (369341, 28))

In [4]:
# # 删去方差为0的特征
# del_cols = ['site_id', 'app_score']
# train_ads = train_ads.drop(columns=del_cols).copy()
# test_ads = test_ads.drop(columns=del_cols).copy()
# train_ads.shape, test_ads.shape

((7675517, 33), (976058, 32))

In [5]:
# # 方便后续拼接
# train_feeds['user_id'] = train_feeds['u_userId']
# test_feeds['user_id'] = test_feeds['u_userId']
# del train_feeds['u_userId'], test_feeds['u_userId']

In [6]:
# # 检查是否用户信息齐全
# (~train_ads.user_id.isin(train_feeds.user_id.values)).sum(), (~test_ads.user_id.isin(test_feeds.user_id.values)).sum()

(0, 0)

In [7]:
# # 保存为pkl格式方便读取
# train_ads.to_pickle('./data/train/train_data_ads.pkl')
# train_feeds.to_pickle('./data/train/train_data_feeds.pkl')

# test_ads.to_pickle('./data/test/test_data_ads.pkl')
# test_feeds.to_pickle('./data/test/test_data_feeds.pkl')

In [1]:
from utils import *

In [2]:
%%time
train_ads, train_feeds = pd.read_pickle('./data/train/train_data_ads.pkl'), pd.read_pickle('./data/train/train_data_feeds.pkl')
test_ads, test_feeds = pd.read_pickle('./data/test/test_data_ads.pkl'), pd.read_pickle('./data/test/test_data_feeds.pkl')
train_ads.shape, train_feeds.shape, test_ads.shape, test_feeds.shape

CPU times: user 1.73 s, sys: 563 ms, total: 2.3 s
Wall time: 2.29 s


((7675517, 33), (3227732, 28), (976058, 32), (369341, 28))

In [3]:
# 先不处理object特征
object_cols = ['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
               'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
               'u_newsCatInterestsST']
train_ads = train_ads.drop(columns=object_cols).copy()
test_ads = test_ads.drop(columns=object_cols).copy()
train_ads.shape, test_ads.shape

((7675517, 26), (976058, 25))

In [4]:
train_features = train_ads.drop(columns=['label']).copy()
train_labels = train_ads['label'].copy()
train_features.shape, train_labels.shape

((7675517, 25), (7675517,))

In [5]:
x_train, x_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=12)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((6140413, 25), (1535104, 25), (6140413,), (1535104,))

In [6]:
x_train_sample = x_train.head(2000).copy()
y_train_sample = y_train.head(2000).copy()
x_val_sample = x_val.head(2000).copy()
y_val_sample = y_val.head(2000).copy()

* 随机森林

In [7]:
%%time
random_base = RandomForestClassifier(random_state=12, n_jobs=14)
random_base.fit(x_train, y_train)

CPU times: user 1h 2min 11s, sys: 16.6 s, total: 1h 2min 27s
Wall time: 4min 38s


RandomForestClassifier(n_jobs=14, random_state=12)

In [9]:
get_res_df(y_val, random_base.predict_proba(x_val)[:, 1], x_val.user_id.values)

,AUC,GAUC,xGAUC
分数,0.764338,0.445309,0.541018


* lgb

In [11]:
%%time
lgb_base = lgb.LGBMClassifier(random_state=12)
lgb_base.fit(x_train, y_train)
get_res_df(y_val, lgb_base.predict_proba(x_val)[:, 1], x_val.user_id.values)

CPU times: user 3min 35s, sys: 558 ms, total: 3min 35s
Wall time: 54.5 s


,AUC,GAUC,xGAUC
分数,0.797443,0.271142,0.429033


In [13]:
%%time
# 用GPU，看看时间
lgb_base = lgb.LGBMClassifier(random_state=12, device='gpu')
lgb_base.fit(x_train, y_train)
get_res_df(y_val, lgb_base.predict_proba(x_val)[:, 1], x_val.user_id.values)

CPU times: user 3min 23s, sys: 969 ms, total: 3min 24s
Wall time: 55.2 s


,AUC,GAUC,xGAUC
分数,0.797593,0.271195,0.429115


* xgb

In [12]:
%%time
xgb_base = xgb.XGBClassifier(random_state=12)
xgb_base.fit(x_train, y_train)
get_res_df(y_val, xgb_base.predict_proba(x_val)[:, 1], x_val.user_id.values)

CPU times: user 24min 29s, sys: 7.52 s, total: 24min 36s
Wall time: 2min 44s


,AUC,GAUC,xGAUC
分数,0.801552,0.271651,0.430621


In [14]:
%%time
# 用GPU，看看时间
xgb_base = xgb.XGBClassifier(random_state=12, tree_method='gpu_hist')
xgb_base.fit(x_train, y_train)
get_res_df(y_val, xgb_base.predict_proba(x_val)[:, 1], x_val.user_id.values)

CPU times: user 56.4 s, sys: 620 ms, total: 57 s
Wall time: 47.4 s


,AUC,GAUC,xGAUC
分数,0.800463,0.271357,0.430089
